In [1]:
# Total least squares assumes corruption in all variables, i.e both in input and output data
#a great way to solve for the solution is by manipulating the k+1th singluar value of the SVD decomposition of [A,b]
import numpy as np
def TLS(A,b):
    svd_matrix= np.vstack((A.T,b)).T
    U,sigma,V_transpose= np.linalg.svd(svd_matrix)
    V= V_transpose.T
    split= A.shape[1]
    Vyy= V[split:,split:]
    Vxy= V[:split,split:]
    sol_tls= -Vxy/Vyy
#now we can also return the denoised A and b as well
    V_split= V[:,split:]
    A_error= np.matmul(svd_matrix,V_split)
    A_error= -np.matmul(A_error,V_split.T)
    A_e= A_error[:,:split]
    A_tls= np.sum(A,A_e)
    b_tls= np.matmul(A_tls,sol_tls)
    return sol_tls,A_tls,b_tls